In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [3]:
from src.models.GAT_HRNN_Multihead import build_model

In [4]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/GAT_HRNN/1k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/integrated_gat_hrnn', 'model': {'type': 'Integrated GAT-HRNN with Dual Attention', 'gat_units': 32, 'lstm_units': 128, 'num_lstm_layers': 2, 'attention_heads': 4, 'integration_units': 256}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5, 'optimizer': 'adam', 'loss': 'categorical_crossentropy'}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'num_agents': 2, 'num_roles': 2, 'path': 'games/tic-tac-toe/1k_single_agent.csv'}, 'input': [{'name': 'agent_features', 'shape': [2, 20]}, {'name': 'adjacency_matrix', 'shape': [2, 2]}, {'name': 'role_masks', 'shape': [2, 2]}], 'output': [{'name': 'event_prediction', 'num_classes': 9}, {'name': 'agent_prediction', 'num_classes': 2}, {'name': 'context_prediction', 'num_classes': 20}], 'evaluation': {'metrics': ['accuracy', 'precision', 'recall', 'f1_score']}, 'visualization': {'attention_maps': True, 'learning_curves': True, 'confusion_matrix': True}}


In [6]:
model_params = {
        'gat_units': config['model']['gat_units'],
        'lstm_units': config['model']['lstm_units'],
        'num_lstm_layers': config['model']['num_lstm_layers'],
        'attention_heads': config['model']['attention_heads'],
       'integration_units': config['model']['integration_units']
    }

In [7]:
# Load and preprocess data
df = load_data(f"../../../../data/processed/{config['data']['path']}")
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model
)


Epoch 1/5


ValueError: Exception encountered when calling TimeDistributed.call().

[1mShape must be rank 2 but is rank 3 for '{{node functional_1_1/time_distributed_1/Tile}} = Tile[T=DT_FLOAT, Tmultiples=DT_INT32](functional_1_1/time_distributed_1/MatMul, functional_1_1/time_distributed_1/Tile/multiples)' with input shapes: [?,32], [3].[0m

Arguments received by TimeDistributed.call():
  • inputs=tf.Tensor(shape=(None, 20, 1), dtype=float32)
  • training=True
  • mask=None

In [ ]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

In [ ]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f'../../../../models/{config['name']}.h5',
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)